In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
from scipy.stats import percentileofscore as score
import math
from statistics import mean
from secrets import IEX_CLOUD_API_TOKEN

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 78704430,
 'calculationPrice': 'iexlasttrade',
 'change': 0.593,
 'changePercent': 0.00405,
 'close': 0,
 'closeSource': 'ficlfoai',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 155.652,
 'iexCloseTime': 1643675588775,
 'iexLastUpdated': 1647271147981,
 'iexMarketPercent': 0.012690190615503838,
 'iexOpen': 154.1,
 'iexOpenTime': 1672045213758,
 'iexRealtimePrice': 153.45,
 'iexRealtimeSize': 1,
 'iexVolume': 742581,
 'lastTradeTime': 1709001125569,
 'latestPrice': 154.96,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'October 20, 2021',
 'latestUpdate': 1635659249183,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'market

In [4]:
pe_ratio = data['peRatio']
pe_ratio



29.37

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy


In [8]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series(
                                        [
                                        symbol,
                                        data[symbol]['quote']['latestPrice'],
                                        data[symbol]['quote']['peRatio'],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index=True)
        
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162.400,49.53,N/A
1,AAL,20.500,-2.0,N/A
2,AAP,233.560,25.2,N/A
3,AAPL,150.419,30.27,N/A
4,ABBV,110.030,30.43,N/A
...,...,...,...,...
500,YUM,126.250,29.43,N/A
501,ZBH,151.910,34.9,N/A
502,ZBRA,551.240,39.06,N/A
503,ZION,65.890,6.65,N/A


In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.390,4.2,N/A
1,BIO,775.800,5.62,N/A
2,MPC,69.910,5.7,N/A
3,PRU,118.000,6.44,N/A
4,ZION,65.890,6.65,N/A
5,GM,59.340,6.75,N/A
6,AFL,56.920,6.9,N/A
7,CINF,121.900,6.95,N/A
8,COO,414.100,7,N/A
9,COF,173.450,7.04,N/A


In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [11]:
portfolio_input()

Enter the value of your portfolio:12345678


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.390,4.2,3149
1,BIO,775.800,5.62,318
2,MPC,69.910,5.7,3531
3,PRU,118.000,6.44,2092
4,ZION,65.890,6.65,3747
5,GM,59.340,6.75,4160
6,AFL,56.920,6.9,4337
7,CINF,121.900,6.95,2025
8,COO,414.100,7,596
9,COF,173.450,7.04,1423


In [13]:
"""
In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
    Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.
"""

"\nIn this section, we will filter for stocks with the lowest percentiles on the following metrics:\n\n    Price-to-earnings ratio\n    Price-to-book ratio\n    Price-to-sales ratio\n    Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)\n    Enterprise Value divided by Gross Profit (EV/GP)\n\nSome of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.\n"

In [14]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [15]:
# rv - robust value
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [16]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,159.230,N/A,51.47,N/A,9.69,N/A,7.97,N/A,30.372804,N/A,14.874070,N/A,N/A
1,AAL,19.770,N/A,-2.0,N/A,-1.69,N/A,0.6785,N/A,-4.618049,N/A,1.888387,N/A,N/A
2,AAP,228.490,N/A,25.0,N/A,4.39,N/A,1.3,N/A,12.397056,N/A,2.898794,N/A,N/A
3,AAPL,155.585,N/A,30.63,N/A,39.49,N/A,7.15,N/A,22.325031,N/A,17.247761,N/A,N/A
4,ABBV,109.500,N/A,30.5,N/A,15.46,N/A,3.7,N/A,10.908653,N/A,7.788735,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.080,N/A,29.93,N/A,-4.85,N/A,5.94,N/A,21.168270,N/A,10.273349,N/A,N/A
501,ZBH,154.270,N/A,34.6,N/A,2.49,N/A,4.1,N/A,19.741312,N/A,6.542902,N/A,N/A
502,ZBRA,528.130,N/A,37.35,N/A,11.4,N/A,5.51,N/A,27.801850,N/A,11.977451,N/A,N/A
503,ZION,65.390,N/A,6.47,N/A,1.42,N/A,3.04,N/A,5.243094,N/A,2.806412,N/A,N/A


In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.90,N/A,60.4,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,321.13,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,292.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.56,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.30,N/A,14.4,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.35,N/A,32.24,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,41.43,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2988.40,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,104.52,N/A,34.93,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [18]:
# for column in rv_dataframe.columns:
#     print(column)

for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
    
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}
for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.849505
1      0.075248
2      0.473267
3       0.60198
4           0.6
         ...   
500    0.594059
501    0.673267
502    0.716832
503    0.089109
504    0.855446
Name: PE Percentile, Length: 505, dtype: object
0       0.79604
1      0.049505
2      0.547525
3      0.960396
4      0.885149
         ...   
500    0.045545
501    0.326733
502    0.827723
503    0.139604
504    0.932673
Name: PB Percentile, Length: 505, dtype: object
0      0.821782
1      0.055446
2      0.161386
3      0.792079
4      0.522772
         ...   
500     0.70495
501    0.567327
502    0.677228
503    0.441584
504    0.912871
Name: PS Percentile, Length: 505, dtype: object
0      0.865347
1      0.025743
2      0.354455
3      0.722772
4       0.29505
         ...   
500     0.70099
501    0.633663
502    0.831683
503    0.055446
504    0.920792
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.825743
1      0.079208
2      0.136634
3      0.871287
4       0.49703
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,159.230,N/A,51.47,0.849505,9.69,0.79604,7.9700,0.821782,30.372804,0.865347,14.874070,0.825743,N/A
1,AAL,19.770,N/A,-2.00,0.075248,-1.69,0.049505,0.6785,0.055446,-4.618049,0.025743,1.888387,0.079208,N/A
2,AAP,228.490,N/A,25.00,0.473267,4.39,0.547525,1.3000,0.161386,12.397056,0.354455,2.898794,0.136634,N/A
3,AAPL,155.585,N/A,30.63,0.60198,39.49,0.960396,7.1500,0.792079,22.325031,0.722772,17.247761,0.871287,N/A
4,ABBV,109.500,N/A,30.50,0.6,15.46,0.885149,3.7000,0.522772,10.908653,0.29505,7.788735,0.49703,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.080,N/A,29.93,0.594059,-4.85,0.045545,5.9400,0.70495,21.168270,0.70099,10.273349,0.669307,N/A
501,ZBH,154.270,N/A,34.60,0.673267,2.49,0.326733,4.1000,0.567327,19.741312,0.633663,6.542902,0.405941,N/A
502,ZBRA,528.130,N/A,37.35,0.716832,11.40,0.827723,5.5100,0.677228,27.801850,0.831683,11.977451,0.744554,N/A
503,ZION,65.390,N/A,6.47,0.089109,1.42,0.139604,3.0400,0.441584,5.243094,0.055446,2.806412,0.126733,N/A


In [20]:
mean([5, 10])

7.5

In [21]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,159.230,N/A,51.47,0.849505,9.69,0.79604,7.9700,0.821782,30.372804,0.865347,14.874070,0.825743,0.831683
1,AAL,19.770,N/A,-2.00,0.075248,-1.69,0.049505,0.6785,0.055446,-4.618049,0.025743,1.888387,0.079208,0.05703
2,AAP,228.490,N/A,25.00,0.473267,4.39,0.547525,1.3000,0.161386,12.397056,0.354455,2.898794,0.136634,0.334653
3,AAPL,155.585,N/A,30.63,0.60198,39.49,0.960396,7.1500,0.792079,22.325031,0.722772,17.247761,0.871287,0.789703
4,ABBV,109.500,N/A,30.50,0.6,15.46,0.885149,3.7000,0.522772,10.908653,0.29505,7.788735,0.49703,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.080,N/A,29.93,0.594059,-4.85,0.045545,5.9400,0.70495,21.168270,0.70099,10.273349,0.669307,0.54297
501,ZBH,154.270,N/A,34.60,0.673267,2.49,0.326733,4.1000,0.567327,19.741312,0.633663,6.542902,0.405941,0.521386
502,ZBRA,528.130,N/A,37.35,0.716832,11.40,0.827723,5.5100,0.677228,27.801850,0.831683,11.977451,0.744554,0.759604
503,ZION,65.390,N/A,6.47,0.089109,1.42,0.139604,3.0400,0.441584,5.243094,0.055446,2.806412,0.126733,0.170495


In [22]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.35,N/A,8.53,0.122772,0.5459,0.051485,0.4281,0.023762,3.614438,0.037624,0.426985,0.009901,0.049109
1,PRU,118.00,N/A,6.43,0.087129,0.7057,0.057426,0.6949,0.057426,4.918377,0.049505,0.681597,0.013861,0.053069
2,AAL,19.77,N/A,-2.00,0.075248,-1.6900,0.049505,0.6785,0.055446,-4.618049,0.025743,1.888387,0.079208,0.05703
3,KSS,47.35,N/A,10.18,0.146535,1.3600,0.121782,0.3965,0.017822,4.007645,0.039604,1.104762,0.033663,0.071881
4,MCK,206.41,N/A,-7.30,0.059406,-60.8500,0.013861,0.1306,0.007921,8.015530,0.164356,2.860607,0.128713,0.074851
5,ALL,129.85,N/A,10.23,0.148515,1.4700,0.151485,0.7743,0.073267,2.489571,0.033663,0.779093,0.017822,0.08495
6,HPQ,29.59,N/A,9.31,0.130693,-8.6200,0.039604,0.5476,0.043564,6.511109,0.093069,2.790003,0.124752,0.086337
7,L,58.95,N/A,10.61,0.160396,0.8291,0.067327,1.0000,0.111881,5.582550,0.063366,0.995086,0.031683,0.086931
8,BA,221.23,N/A,-14.40,0.053465,-7.9600,0.041584,2.1300,0.328713,-30.633508,0.007921,-207.673534,0.00396,0.087129
9,RE,288.60,N/A,8.43,0.120792,1.0800,0.079208,0.9879,0.108911,7.032823,0.116832,0.980819,0.029703,0.091089


In [23]:
portfolio_input()

Enter the value of your portfolio:12345678


In [24]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

In [25]:
position_size

246913.56

In [26]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

Z:\Projects\strategy_index_screener\.venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.35,8412,8.53,0.122772,0.5459,0.051485,0.4281,0.023762,3.614438,0.037624,0.426985,0.009901,0.049109
1,PRU,118.00,2092,6.43,0.087129,0.7057,0.057426,0.6949,0.057426,4.918377,0.049505,0.681597,0.013861,0.053069
2,AAL,19.77,12489,-2.00,0.075248,-1.6900,0.049505,0.6785,0.055446,-4.618049,0.025743,1.888387,0.079208,0.05703
3,KSS,47.35,5214,10.18,0.146535,1.3600,0.121782,0.3965,0.017822,4.007645,0.039604,1.104762,0.033663,0.071881
4,MCK,206.41,1196,-7.30,0.059406,-60.8500,0.013861,0.1306,0.007921,8.015530,0.164356,2.860607,0.128713,0.074851
5,ALL,129.85,1901,10.23,0.148515,1.4700,0.151485,0.7743,0.073267,2.489571,0.033663,0.779093,0.017822,0.08495
6,HPQ,29.59,8344,9.31,0.130693,-8.6200,0.039604,0.5476,0.043564,6.511109,0.093069,2.790003,0.124752,0.086337
7,L,58.95,4188,10.61,0.160396,0.8291,0.067327,1.0000,0.111881,5.582550,0.063366,0.995086,0.031683,0.086931
8,BA,221.23,1116,-14.40,0.053465,-7.9600,0.041584,2.1300,0.328713,-30.633508,0.007921,-207.673534,0.00396,0.087129
9,RE,288.60,855,8.43,0.120792,1.0800,0.079208,0.9879,0.108911,7.032823,0.116832,0.980819,0.029703,0.091089


In [27]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)



In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [30]:
writer.save()